Since we are done with collecting the data from the given site. We must clean the data to get rid of unnecessary informaation.

In [23]:
pip install nltk

     ---------------------------------------- 1.5/1.5 MB 7.4 MB/s eta 0:00:00
     ---------------------------------------- 78.3/78.3 KB 4.3 MB/s eta 0:00:00
     ------------------------------------- 269.6/269.6 KB 16.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\admin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [29]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import nltk


In [33]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...


True

In [31]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

Loading the csv file "BA_reviews2.csv" into a dataframe.

In [2]:
df=pd.read_csv("C:/python(me)/british-airways/BA_reviews2.csv")

In [3]:
df.head(5)

,Unnamed: 0,reviews,stars,date,country
0,0,✅ Trip Verified | The customer services (call ...,5,14th October 2023,United Kingdom
1,1,✅ Trip Verified | I am a frequent flyer with...,3,14th October 2023,United Kingdom
2,2,Not Verified | Flew with BA to Punta Cana. To...,5,12th October 2023,United Kingdom
3,3,✅ Trip Verified | They downgraded me from bu...,4,12th October 2023,Australia
4,4,✅ Trip Verified | The already delayed aircra...,1,11th October 2023,United Kingdom


In [42]:
df.shape

(1000, 6)

In [40]:

df.columns

Index(['Unnamed: 0', 'reviews', 'stars', 'date', 'country', 'verified',
       'corpus'],
      dtype='object')

In [41]:
del df['Unnamed: 0']
df.reset_index(drop=True)

,reviews,stars,date,country,verified,corpus
0,✅ Trip Verified | The customer services (call ...,5,2023-10-14,United Kingdom,True,customer service call centre etc atrocious som...
1,✅ Trip Verified | I am a frequent flyer with...,3,2023-10-14,United Kingdom,True,frequent flyer ba last year noted rating given...
2,Not Verified | Flew with BA to Punta Cana. To...,5,2023-10-12,United Kingdom,False,verified flew ba punta cana took trouble reser...
3,✅ Trip Verified | They downgraded me from bu...,4,2023-10-12,Australia,True,hey downgraded business premium economy took t...
4,✅ Trip Verified | The already delayed aircra...,1,2023-10-11,United Kingdom,True,already delayed aircraft landed apart small ba...
...,...,...,...,...,...,...
995,✅ Trip Verified | Los Angeles to London Heath...,7,2018-12-23,United States,True,los angeles london heathrow poor customer serv...
996,✅ Trip Verified | Our flight from Seattle to ...,1,2018-12-23,United States,True,flight seattle london november th canceled due...
997,✅ Trip Verified | Gatwick to Barbados. We fle...,10,2018-12-22,United Kingdom,True,gatwick barbados flew gatwick th december back...
998,✅ Trip Verified | Moscow to London. Fantastic...,2,2018-12-19,United Kingdom,True,moscow london fantastic first class service mo...


Looking at our data to glean some info

Check if there are any null values

In [4]:
df.isnull().sum()

Unnamed: 0    0
reviews       0
stars         0
date          0
country       0
dtype: int64

Checking the datatype for each columns

In [12]:
df.dtypes

Unnamed: 0     int64
reviews       object
stars          int64
date          object
country       object
dtype: object

Cleaning the stars Column first

In [10]:
df.stars.value_counts()

1     348
2     119
10     98
3      93
9      77
8      70
5      58
4      56
7      45
6      36
Name: stars, dtype: int64

Formating the date column

In [14]:
df.date=pd.to_datetime(df.date)
df.date

0     2023-10-14
1     2023-10-14
2     2023-10-12
3     2023-10-12
4     2023-10-11
         ...    
995   2018-12-23
996   2018-12-23
997   2018-12-22
998   2018-12-19
999   2018-12-19
Name: date, Length: 1000, dtype: datetime64[ns]

In [15]:
df.head(5)

,Unnamed: 0,reviews,stars,date,country
0,0,✅ Trip Verified | The customer services (call ...,5,2023-10-14,United Kingdom
1,1,✅ Trip Verified | I am a frequent flyer with...,3,2023-10-14,United Kingdom
2,2,Not Verified | Flew with BA to Punta Cana. To...,5,2023-10-12,United Kingdom
3,3,✅ Trip Verified | They downgraded me from bu...,4,2023-10-12,Australia
4,4,✅ Trip Verified | The already delayed aircra...,1,2023-10-11,United Kingdom


Cleaning the reviews column

In [34]:
#for lemmatization of words we will use nltk library
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemma = WordNetLemmatizer()

#Removing the trip verified part by inserting it into a new column incase we need the info later 
df['verified'] = df.reviews.str.contains("Trip Verified")
review_d=df.reviews.str.strip("✅ Trip Verified |")

#create an empty list to collect cleaned data corpus
corpus =[]

#loop through each review, remove punctuations, small case it, join it and add it to corpus
for rev in review_d:
    rev = re.sub('[^a-zA-Z]',' ', rev)
    rev = rev.lower()
    rev = rev.split()
    rev = [lemma.lemmatize(word) for word in rev if word not in set(stopwords.words("english"))]
    rev = " ".join(rev)
    corpus.append(rev)

In [35]:
df['corpus'] = corpus

In [36]:
df.head(5)

,Unnamed: 0,reviews,stars,date,country,verified,corpus
0,0,✅ Trip Verified | The customer services (call ...,5,2023-10-14,United Kingdom,True,customer service call centre etc atrocious som...
1,1,✅ Trip Verified | I am a frequent flyer with...,3,2023-10-14,United Kingdom,True,frequent flyer ba last year noted rating given...
2,2,Not Verified | Flew with BA to Punta Cana. To...,5,2023-10-12,United Kingdom,False,verified flew ba punta cana took trouble reser...
3,3,✅ Trip Verified | They downgraded me from bu...,4,2023-10-12,Australia,True,hey downgraded business premium economy took t...
4,4,✅ Trip Verified | The already delayed aircra...,1,2023-10-11,United Kingdom,True,already delayed aircraft landed apart small ba...


In [43]:
df.to_csv("C:/python(me)/british-airways/cleaned-BA-reviews.csv")